In [13]:
# 01_data_exploration_and_feature_engineering.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
import os
warnings.filterwarnings('ignore')


In [14]:

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("=" * 80)
print("HEALTHCARE PROVIDER FRAUD DETECTION - DATA EXPLORATION")
print("=" * 80)

# Create directories if they don't exist
os.makedirs('../reports/figures', exist_ok=True)
os.makedirs('../models', exist_ok=True)

HEALTHCARE PROVIDER FRAUD DETECTION - DATA EXPLORATION


# 0. Download Data

# 1. LOAD DATA

In [15]:

print("\n1. LOADING DATASETS...")

# Load all data from training files only
beneficiary_data = pd.read_csv('../data/Train_Beneficiarydata-1542865627584.csv')
inpatient_data = pd.read_csv('../data/Train_Inpatientdata-1542865627584.csv')
outpatient_data = pd.read_csv('../data/Train_Outpatientdata-1542865627584.csv')
labels_data = pd.read_csv('../data/Train-1542865627584.csv')

print(f"✓ Beneficiary Data: {beneficiary_data.shape}")
print(f"✓ Inpatient Data: {inpatient_data.shape}")
print(f"✓ Outpatient Data: {outpatient_data.shape}")
print(f"✓ Labels Data: {labels_data.shape}")

# Check fraud distribution in full dataset
print(f"\n--- Full Dataset Fraud Distribution ---")
print(labels_data['PotentialFraud'].value_counts())
fraud_pct = (labels_data['PotentialFraud'] == 'Yes').sum() / len(labels_data) * 100
print(f"Fraud Percentage: {fraud_pct:.2f}%")



1. LOADING DATASETS...


✓ Beneficiary Data: (138556, 25)
✓ Inpatient Data: (40474, 30)
✓ Outpatient Data: (517737, 27)
✓ Labels Data: (5410, 2)

--- Full Dataset Fraud Distribution ---
PotentialFraud
No     4904
Yes     506
Name: count, dtype: int64
Fraud Percentage: 9.35%


# 2. DATA STRUCTURE ANALYSIS

In [16]:
print("\n2. UNDERSTANDING DATA STRUCTURE...")

print("\n--- Beneficiary Data Sample ---")
print(beneficiary_data.head(3))
print(f"\nColumns ({len(beneficiary_data.columns)}): {beneficiary_data.columns.tolist()}")

print("\n--- Inpatient Data Sample ---")
print(inpatient_data.head(3))
print(f"\nColumns ({len(inpatient_data.columns)}): {inpatient_data.columns.tolist()}")

print("\n--- Outpatient Data Sample ---")
print(outpatient_data.head(3))
print(f"\nColumns ({len(outpatient_data.columns)}): {outpatient_data.columns.tolist()}")

print("\n--- Labels Sample ---")
print(labels_data.head())
print(f"\nColumns: {labels_data.columns.tolist()}")



2. UNDERSTANDING DATA STRUCTURE...

--- Beneficiary Data Sample ---
      BeneID         DOB  DOD  Gender  Race RenalDiseaseIndicator  State  \
0  BENE11001  1943-01-01  NaN       1     1                     0     39   
1  BENE11002  1936-09-01  NaN       2     1                     0     39   
2  BENE11003  1936-08-01  NaN       1     1                     0     52   

   County  NoOfMonths_PartACov  NoOfMonths_PartBCov  ...  \
0     230                   12                   12  ...   
1     280                   12                   12  ...   
2     590                   12                   12  ...   

   ChronicCond_Depression  ChronicCond_Diabetes  ChronicCond_IschemicHeart  \
0                       1                     1                          1   
1                       2                     2                          2   
2                       2                     2                          1   

   ChronicCond_Osteoporasis  ChronicCond_rheumatoidarthritis  \
0       


# 3. DATA QUALITY ASSESSMENT


In [17]:

print("\n3. DATA QUALITY ASSESSMENT...")

def assess_quality(df, name):
    print(f"\n--- {name} ---")
    print(f"Shape: {df.shape}")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"Missing values:\n{missing[missing > 0]}")
    else:
        print("No missing values")
    print(f"Duplicates: {df.duplicated().sum()}")

assess_quality(beneficiary_data, "Beneficiary Data")
assess_quality(inpatient_data, "Inpatient Data")
assess_quality(outpatient_data, "Outpatient Data")
assess_quality(labels_data, "Labels Data")



3. DATA QUALITY ASSESSMENT...

--- Beneficiary Data ---
Shape: (138556, 25)
Missing values:
DOD    137135
dtype: int64
Duplicates: 0

--- Inpatient Data ---
Shape: (40474, 30)
Missing values:
AttendingPhysician       112
OperatingPhysician     16644
OtherPhysician         35784
DeductibleAmtPaid        899
ClmDiagnosisCode_2       226
ClmDiagnosisCode_3       676
ClmDiagnosisCode_4      1534
ClmDiagnosisCode_5      2894
ClmDiagnosisCode_6      4838
ClmDiagnosisCode_7      7258
ClmDiagnosisCode_8      9942
ClmDiagnosisCode_9     13497
ClmDiagnosisCode_10    36547
ClmProcedureCode_1     17326
ClmProcedureCode_2     35020
ClmProcedureCode_3     39509
ClmProcedureCode_4     40358
ClmProcedureCode_5     40465
ClmProcedureCode_6     40474
dtype: int64
Duplicates: 0

--- Outpatient Data ---
Shape: (517737, 27)
Missing values:
AttendingPhysician         1396
OperatingPhysician       427120
OtherPhysician           322691
ClmDiagnosisCode_1        10453
ClmDiagnosisCode_2       195380
ClmDiagn



# 4. KEY RELATIONSHIP ANALYSIS


In [18]:
print("\n4. ANALYZING RELATIONSHIPS...")

print(f"Unique Beneficiaries: {beneficiary_data['BeneID'].nunique()}")
print(f"Unique Providers in Inpatient: {inpatient_data['Provider'].nunique()}")
print(f"Unique Providers in Outpatient: {outpatient_data['Provider'].nunique()}")
print(f"Unique Providers in Labels: {labels_data['Provider'].nunique()}")
print(f"Avg Inpatient claims/provider: {len(inpatient_data) / inpatient_data['Provider'].nunique():.2f}")
print(f"Avg Outpatient claims/provider: {len(outpatient_data) / outpatient_data['Provider'].nunique():.2f}")



4. ANALYZING RELATIONSHIPS...
Unique Beneficiaries: 138556
Unique Providers in Inpatient: 2092
Unique Providers in Outpatient: 5012
Unique Providers in Labels: 5410
Avg Inpatient claims/provider: 19.35
Avg Outpatient claims/provider: 103.30



# 5. FEATURE ENGINEERING FUNCTIONS


In [19]:

print("\n5. DEFINING FEATURE ENGINEERING FUNCTIONS...")

def create_beneficiary_features(bene_df):
    """Aggregate beneficiary characteristics"""
    
    features = pd.DataFrame()
    features['BeneID'] = bene_df['BeneID']
    
    # Demographics
    if 'Gender' in bene_df.columns:
        features['Gender'] = bene_df['Gender']
    
    if 'Race' in bene_df.columns:
        features['Race'] = bene_df['Race']
    
    # Age calculation
    if 'DOB' in bene_df.columns:
        bene_df['DOB'] = pd.to_datetime(bene_df['DOB'], errors='coerce')
        reference_date = pd.to_datetime('2009-12-01')
        features['Age'] = (reference_date - bene_df['DOB']).dt.days / 365.25
    
    # Count chronic conditions
    chronic_cols = [col for col in bene_df.columns if 'Chronic' in col]
    if chronic_cols:
        features['ChronicConditionsCount'] = bene_df[chronic_cols].sum(axis=1)
    
    # Renal disease indicator
    if 'RenalDiseaseIndicator' in bene_df.columns:
        features['HasRenalDisease'] = bene_df['RenalDiseaseIndicator'].map({'Y': 1, '0': 0}).fillna(0)
    
    # Is deceased
    if 'DOD' in bene_df.columns:
        features['IsDeceased'] = bene_df['DOD'].notna().astype(int)
    
    return features

def aggregate_claims(claims_df, claim_type):
    """Aggregate claim-level data to provider level"""
    
    provider_features = []
    
    for provider, group in claims_df.groupby('Provider'):
        features = {'Provider': provider}
        
        # Basic claim statistics
        features[f'{claim_type}_NumClaims'] = len(group)
        features[f'{claim_type}_NumBeneficiaries'] = group['BeneID'].nunique()
        features[f'{claim_type}_AvgClaimsPerBeneficiary'] = len(group) / group['BeneID'].nunique()
        
        # Financial features
        if 'InscClaimAmtReimbursed' in group.columns:
            features[f'{claim_type}_TotalReimbursed'] = group['InscClaimAmtReimbursed'].sum()
            features[f'{claim_type}_AvgReimbursed'] = group['InscClaimAmtReimbursed'].mean()
            features[f'{claim_type}_StdReimbursed'] = group['InscClaimAmtReimbursed'].std()
            features[f'{claim_type}_MaxReimbursed'] = group['InscClaimAmtReimbursed'].max()
            features[f'{claim_type}_MinReimbursed'] = group['InscClaimAmtReimbursed'].min()
        
        if 'DeductibleAmtPaid' in group.columns:
            features[f'{claim_type}_TotalDeductible'] = group['DeductibleAmtPaid'].sum()
            features[f'{claim_type}_AvgDeductible'] = group['DeductibleAmtPaid'].mean()
        
        # Physician statistics
        physician_cols = [col for col in group.columns if 'Physician' in col]
        if physician_cols:
            features[f'{claim_type}_NumUniquePhysicians'] = group[physician_cols].nunique().sum()
        
        # Diagnosis codes
        diag_cols = [col for col in group.columns if 'ClmDiagnosisCode' in col]
        if diag_cols:
            unique_diagnoses = set()
            for col in diag_cols:
                unique_diagnoses.update(group[col].dropna().unique())
            features[f'{claim_type}_UniqueDiagnoses'] = len(unique_diagnoses)
        
        # Procedure codes
        proc_cols = [col for col in group.columns if 'ClmProcedureCode' in col]
        if proc_cols:
            unique_procedures = set()
            for col in proc_cols:
                unique_procedures.update(group[col].dropna().unique())
            features[f'{claim_type}_UniqueProcedures'] = len(unique_procedures)
        
        # Admission duration (for inpatient only)
        if claim_type == 'Inpatient' and 'AdmissionDt' in group.columns and 'DischargeDt' in group.columns:
            group['AdmissionDt'] = pd.to_datetime(group['AdmissionDt'], errors='coerce')
            group['DischargeDt'] = pd.to_datetime(group['DischargeDt'], errors='coerce')
            group['LOS'] = (group['DischargeDt'] - group['AdmissionDt']).dt.days
            features[f'{claim_type}_AvgLOS'] = group['LOS'].mean()
            features[f'{claim_type}_TotalLOS'] = group['LOS'].sum()
            features[f'{claim_type}_MaxLOS'] = group['LOS'].max()
        
        # Claim dates analysis
        if 'ClaimStartDt' in group.columns and 'ClaimEndDt' in group.columns:
            group['ClaimStartDt'] = pd.to_datetime(group['ClaimStartDt'], errors='coerce')
            group['ClaimEndDt'] = pd.to_datetime(group['ClaimEndDt'], errors='coerce')
            group['ClaimDuration'] = (group['ClaimEndDt'] - group['ClaimStartDt']).dt.days
            features[f'{claim_type}_AvgClaimDuration'] = group['ClaimDuration'].mean()
        
        provider_features.append(features)
    
    return pd.DataFrame(provider_features)

def aggregate_bene_by_provider(claims_df, bene_features):
    """Aggregate beneficiary characteristics at provider level"""
    
    # Merge claims with beneficiary features
    claims_with_bene = claims_df.merge(bene_features, on='BeneID', how='left')
    
    # Aggregate by provider
    agg_dict = {}
    
    if 'Age' in claims_with_bene.columns:
        agg_dict['Age'] = ['mean', 'std', 'min', 'max']
    if 'Gender' in claims_with_bene.columns:
        agg_dict['Gender'] = 'mean'
    if 'ChronicConditionsCount' in claims_with_bene.columns:
        agg_dict['ChronicConditionsCount'] = ['mean', 'max', 'sum']
    if 'IsDeceased' in claims_with_bene.columns:
        agg_dict['IsDeceased'] = 'sum'
    if 'HasRenalDisease' in claims_with_bene.columns:
        agg_dict['HasRenalDisease'] = 'sum'
    
    if not agg_dict:
        return pd.DataFrame({'Provider': claims_df['Provider'].unique()})
    
    provider_bene = claims_df.merge(bene_features, on='BeneID', how='left')\
                              .groupby('Provider').agg(agg_dict).reset_index()
    
    # Flatten column names
    provider_bene.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                             for col in provider_bene.columns.values]
    
    return provider_bene



5. DEFINING FEATURE ENGINEERING FUNCTIONS...




# 6. PROCESS TRAINING DATA


In [20]:

print("\n6. PROCESSING DATA AND CREATING FEATURES...")

print("Creating beneficiary features...")
bene_features = create_beneficiary_features(beneficiary_data)

print("Aggregating inpatient claims...")
inpatient_features = aggregate_claims(inpatient_data, 'Inpatient')

print("Aggregating outpatient claims...")
outpatient_features = aggregate_claims(outpatient_data, 'Outpatient')

print("Aggregating beneficiary info by provider (inpatient)...")
inpatient_bene = aggregate_bene_by_provider(inpatient_data, bene_features)

print("Aggregating beneficiary info by provider (outpatient)...")
outpatient_bene = aggregate_bene_by_provider(outpatient_data, bene_features)

# Merge all features
print("Merging all provider features...")
provider_data = labels_data.copy()
provider_data = provider_data.merge(inpatient_features, on='Provider', how='left')
provider_data = provider_data.merge(outpatient_features, on='Provider', how='left')
provider_data = provider_data.merge(inpatient_bene, on='Provider', how='left')
provider_data = provider_data.merge(outpatient_bene, on='Provider', how='left', 
                                    suffixes=('_Inp', '_Out'))

# Fill NaN (providers with no claims in one category)
provider_data = provider_data.fillna(0)

print(f"\n✓ Full provider dataset: {provider_data.shape}")
print(f"✓ Features created: {provider_data.shape[1] - 2}")



6. PROCESSING DATA AND CREATING FEATURES...
Creating beneficiary features...
Aggregating inpatient claims...


Aggregating outpatient claims...
Aggregating beneficiary info by provider (inpatient)...
Aggregating beneficiary info by provider (outpatient)...
Aggregating beneficiary info by provider (inpatient)...
Aggregating beneficiary info by provider (outpatient)...
Merging all provider features...

✓ Full provider dataset: (5410, 53)
✓ Features created: 51
Merging all provider features...

✓ Full provider dataset: (5410, 53)
✓ Features created: 51



# 7. SPLIT DATA INTO TRAIN AND TEST SETS


In [21]:

print("\n7. SPLITTING DATA INTO TRAIN AND TEST SETS...")

from sklearn.model_selection import train_test_split

# Split provider data into train and test (80/20 split)
train_provider_data, test_provider_data = train_test_split(
    provider_data, 
    test_size=0.2, 
    random_state=42, 
    stratify=provider_data['PotentialFraud']
)

print(f"\n✓ Training provider dataset: {train_provider_data.shape}")
print(f"✓ Test provider dataset: {test_provider_data.shape}")

# Check fraud distribution in both sets
train_fraud_counts = train_provider_data['PotentialFraud'].value_counts()
test_fraud_counts = test_provider_data['PotentialFraud'].value_counts()

print(f"\n--- Training Set Distribution ---")
print(train_fraud_counts)
print(f"Fraud Percentage: {(train_fraud_counts.get('Yes', 0) / len(train_provider_data) * 100):.2f}%")

print(f"\n--- Test Set Distribution ---")
print(test_fraud_counts)
print(f"Fraud Percentage: {(test_fraud_counts.get('Yes', 0) / len(test_provider_data) * 100):.2f}%")



7. SPLITTING DATA INTO TRAIN AND TEST SETS...

✓ Training provider dataset: (4328, 53)
✓ Test provider dataset: (1082, 53)

--- Training Set Distribution ---
PotentialFraud
No     3923
Yes     405
Name: count, dtype: int64
Fraud Percentage: 9.36%

--- Test Set Distribution ---
PotentialFraud
No     981
Yes    101
Name: count, dtype: int64
Fraud Percentage: 9.33%




# 8. EXPLORATORY DATA ANALYSIS


In [22]:

print("\n8. EXPLORATORY DATA ANALYSIS...")

# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

train_fraud_counts = train_provider_data['PotentialFraud'].value_counts()
axes[0].bar(train_fraud_counts.index, train_fraud_counts.values, color=['green', 'red'])
axes[0].set_title('Training Set - Fraud Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Potential Fraud')
axes[0].set_ylabel('Count')
for i, v in enumerate(train_fraud_counts.values):
    axes[0].text(i, v + 20, str(v), ha='center', fontweight='bold')

test_fraud_counts = test_provider_data['PotentialFraud'].value_counts()
axes[1].bar(test_fraud_counts.index, test_fraud_counts.values, color=['green', 'red'])
axes[1].set_title('Test Set - Fraud Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Potential Fraud')
axes[1].set_ylabel('Count')
for i, v in enumerate(test_fraud_counts.values):
    axes[1].text(i, v + 20, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/figures/target_distribution.png', dpi=300, bbox_inches='tight')
print("✓ Saved: target_distribution.png")
plt.close()

# Fraud comparison - key metrics
fraud_yes = train_provider_data[train_provider_data['PotentialFraud'] == 'Yes']
fraud_no = train_provider_data[train_provider_data['PotentialFraud'] == 'No']

numeric_cols = train_provider_data.select_dtypes(include=[np.number]).columns.tolist()

# Select key comparison metrics
comparison_metrics = [col for col in [
    'Inpatient_TotalReimbursed', 'Outpatient_TotalReimbursed',
    'Inpatient_NumClaims', 'Outpatient_NumClaims',
    'Age_mean_Inp', 'IsDeceased_sum_Inp'
] if col in numeric_cols]

if len(comparison_metrics) >= 6:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    for i, col in enumerate(comparison_metrics[:6]):
        axes[i].boxplot([fraud_no[col].dropna(), fraud_yes[col].dropna()], 
                       labels=['No Fraud', 'Fraud'])
        axes[i].set_title(col, fontweight='bold')
        axes[i].set_ylabel('Value')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../reports/figures/fraud_comparison.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: fraud_comparison.png")
    plt.close()

# Correlation heatmap
if len(numeric_cols) > 1:
    plt.figure(figsize=(12, 10))
    correlation_matrix = train_provider_data[numeric_cols].corr()
    n_features = min(20, len(numeric_cols))
    sns.heatmap(correlation_matrix.iloc[:n_features, :n_features], 
               annot=False, cmap='coolwarm', center=0)
    plt.title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../reports/figures/correlation_heatmap.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: correlation_heatmap.png")
    plt.close()



8. EXPLORATORY DATA ANALYSIS...


✓ Saved: target_distribution.png
✓ Saved: fraud_comparison.png
✓ Saved: fraud_comparison.png
✓ Saved: correlation_heatmap.png
✓ Saved: correlation_heatmap.png




# 9. SAVE PROCESSED DATA


In [23]:

print("\n9. SAVING PROCESSED DATA...")

train_provider_data.to_csv('../data/processed_train_provider_data.csv', index=False)
test_provider_data.to_csv('../data/processed_test_provider_data.csv', index=False)

print("✓ Saved: processed_train_provider_data.csv")
print("✓ Saved: processed_test_provider_data.csv")

print("\n" + "=" * 80)
print("DATA EXPLORATION COMPLETE!")
print("=" * 80)
print(f"\nFull Dataset Summary:")
print(f"  Total Providers: {len(provider_data)}")
print(f"  Total Features: {provider_data.shape[1] - 2}")

print(f"\nTraining Dataset Summary:")
print(f"  Total Providers: {len(train_provider_data)}")
print(f"  Fraudulent: {len(fraud_yes)} ({len(fraud_yes)/len(train_provider_data)*100:.2f}%)")
print(f"  Legitimate: {len(fraud_no)} ({len(fraud_no)/len(train_provider_data)*100:.2f}%)")
print(f"  Total Features: {len(numeric_cols)}")

print(f"\nTest Dataset Summary:")
print(f"  Total Providers: {len(test_provider_data)}")
print(f"  Fraudulent: {test_fraud_counts.get('Yes', 0)} ({test_fraud_counts.get('Yes', 0)/len(test_provider_data)*100:.2f}%)")
print(f"  Legitimate: {test_fraud_counts.get('No', 0)} ({test_fraud_counts.get('No', 0)/len(test_provider_data)*100:.2f}%)")
print(f"  Total Features: {test_provider_data.shape[1] - 2}")

print(f"\nNext Step: Run 02_modeling.ipynb")



9. SAVING PROCESSED DATA...
✓ Saved: processed_train_provider_data.csv
✓ Saved: processed_test_provider_data.csv

DATA EXPLORATION COMPLETE!

Full Dataset Summary:
  Total Providers: 5410
  Total Features: 51

Training Dataset Summary:
  Total Providers: 4328
  Fraudulent: 405 (9.36%)
  Legitimate: 3923 (90.64%)
  Total Features: 51

Test Dataset Summary:
  Total Providers: 1082
  Fraudulent: 101 (9.33%)
  Legitimate: 981 (90.67%)
  Total Features: 51

Next Step: Run 02_modeling.ipynb
✓ Saved: processed_train_provider_data.csv
✓ Saved: processed_test_provider_data.csv

DATA EXPLORATION COMPLETE!

Full Dataset Summary:
  Total Providers: 5410
  Total Features: 51

Training Dataset Summary:
  Total Providers: 4328
  Fraudulent: 405 (9.36%)
  Legitimate: 3923 (90.64%)
  Total Features: 51

Test Dataset Summary:
  Total Providers: 1082
  Fraudulent: 101 (9.33%)
  Legitimate: 981 (90.67%)
  Total Features: 51

Next Step: Run 02_modeling.ipynb
